In [54]:
import numpy as np
import random

In [55]:
class Module():
    def __init__(self,module):
        module = module.split('|')
        self.name = module[0]
        self.staff = module[1]
        self.labs = module[2]
        self.constraints = module[3].split(',')
        
    def isin(self,mod):
        p

f = open('Modules.txt','r')
modules = []
line = f.readline()
while line:
    modules.append(Module(line))
    line = f.readline()
f.close()

In [119]:
timetable = np.full((20,3),Module("None|None|None|None,None"),dtype=Module)

totalmodules = modules
for i in modules:
    totalmodules = totalmodules + ([i]*int(i.labs))
k = 0
for i in range(len(timetable)):
    for j in range(len(timetable[i])):
        timetable[i][j] = totalmodules[k]
        k = k+1
        if k>=len(totalmodules):
            break
    if k>=len(totalmodules):
            break
            

timetable = timetable.flatten()
np.random.shuffle(timetable)
timetable = np.reshape(timetable,(20,3))

timetable

array([[<__main__.Module object at 0x7f9628aa9b20>,
        <__main__.Module object at 0x7f9628aa92b0>]], dtype=object)

In [ ]:
def generate_random_timetable():
    timetable = timetable.flatten()
    np.random.shuffle(timetable)
    timetable = np.reshape(timetable,(20,3))
    return timetable

def 

In [118]:
def checkConcurrence(li):
    count = 0
    #Checks if module is against another module that clashes.
    if li[0].name in li[1].constraints or li[0].name in li[2].constraints:
        count = count + 1
    if li[1].name in li[2].constraints or li[1].name in li[0].constraints:
        count = count + 1
    if li[2].name in li[0].constraints or li[2].name in li[1].constraints:
        count = count + 1
    
    #Checks if one module is being held at the same time in different rooms.
    if li[0].name==li[1].name:
        count = count + 1
    if li[1].name==li[2].name:
        count = count + 1
    if li[2].name==li[0].name:
        count = count + 1
    
    return count

def checkPrecedence(li):
    count = 0
    for i in range(len(li)):
        lecture = li[i][0].name
        if lecture == None:
            continue
        for j in range(0,i):
            if lecture == li[j][1].name:
                count = count + 1
            if lecture == li[j][2].name:
                count = count + 1
    return count


def fitness(timetable):
    precedence = checkPrecedence(timetable)
    
    concurrence = 0
    for i in timetable:
        concurrence = concurrence + checkConcurrence(i)
    
    return precedence * concurrence
        
print(fitness(timetable))

490


In [ ]:
def hillclimber(Niter, problem, D, generate, mutation, comparison):
    #comparison is to compare fitness
    #problem is fitness
    parent = generate(D)
    parent_fitness = problem(parent)
    
    for i in range(Niter):
        #Mutate
        child = mutation(parent)
        
        #Evaluate
        
        \\
        #Pick next parent solution
    